# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    # UPDATE KKO - convert newlines to spaces
    text=f.read().replace('\n', ' ')
# UPDATE KKO - omit newlines
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1   Happy families are all alike; every unhappy family is unhappy in its own way.  Everythin'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([24, 57, 50, 65, 69, 54, 67,  0, 10,  0,  0,  0, 29, 50, 65, 65, 74,
        0, 55, 50, 62, 58, 61, 58, 54, 68,  0, 50, 67, 54,  0, 50, 61, 61,
        0, 50, 61, 58, 60, 54, 20,  0, 54, 71, 54, 67, 74,  0, 70, 63, 57,
       50, 65, 65, 74,  0, 55, 50, 62, 58, 61, 74,  0, 58, 68,  0, 70, 63,
       57, 50, 65, 65, 74,  0, 58, 63,  0, 58, 69, 68,  0, 64, 72, 63,  0,
       72, 50, 74,  8,  0,  0, 26, 71, 54, 67, 74, 69, 57, 58, 63], dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

76

In [6]:
print(','.join(vocab))

 ,!,",',(,),,,-,.,0,1,2,3,4,5,6,7,8,9,:,;,?,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,_,`,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z


## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

We start with our text encoded as integers in one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`batch_size` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. 

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [94]:
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = batch_size * n_steps
    n_batches = len(arr) // characters_per_batch
    n_tot_characters = characters_per_batch * n_batches
    
    # Keep only enough characters to make full batches
    arr = arr[:n_tot_characters]
    
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:(n+n_steps)]
        # The targets, shifted by one
        # y = arr[:, (n+1):(n+1+n_steps)]
        y = np.roll(x, -1)
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [95]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [96]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[24 57 50 65 69 54 67  0 10  0]
 [52 60  0 57 54 67  6  0 64 55]
 [ 0 50 69  0 69 57 54  0 68 50]
 [69 57 54 67  3 68  0 67 58 56]
 [21  0 23 74  0 52 57 50 63 52]
 [50 53 54 62 64 58 68 54 61 61]
 [61 50 62 54  0 68 64 62 54 64]
 [ 0 40 54 67 56 54 74  0 30 71]
 [57 54 62  0 52 64 62 65 61 54]
 [57  0 50  0 61 54 69 69 54 67]]

y
 [[57 50 65 69 54 67  0 10  0  0]
 [60  0 57 54 67  6  0 64 55  0]
 [50 69  0 69 57 54  0 68 50 62]
 [57 54 67  3 68  0 67 58 56 57]
 [ 0 23 74  0 52 57 50 63 52 54]
 [53 54 62 64 58 68 54 61 61 54]
 [50 62 54  0 68 64 62 54 64 63]
 [40 54 67 56 54 74  0 30 71 50]
 [54 62  0 52 64 62 65 61 54 69]
 [ 0 50  0 61 54 69 69 54 67  8]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [48]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, shape = (batch_size, num_steps))
    targets = tf.placeholder(tf.int32, shape = (batch_size, num_steps))
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32)
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [49]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units = lstm_size)
    
    # Add dropout to the cell outputs
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob = keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [61]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis = 1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal([in_size, out_size]))
        softmax_b = tf.Variable(tf.zeros([out_size]))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.add(tf.matmul(x, softmax_w), softmax_b)
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits)
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [78]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, depth = num_classes, axis = -1)
    y_reshaped =  tf.reshape(tf.concat(y_one_hot, axis = 1), [-1, num_classes])
    # Could also just use tf.reshape and also use logits.get_shape() as the shape
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(labels = y_reshaped, logits = logits)
    loss = tf.reduce_mean(loss)
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [79]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [80]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, depth = num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state = self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss =  build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [81]:
batch_size = 10         # Sequences per batch
num_steps = 50          # Number of sequence steps per batch
lstm_size = 128         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.01    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [97]:
epochs = 20
# Print losses every N interations
print_every_n = 50

# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 50...  Training loss: 2.9489...  0.0556 sec/batch
Epoch: 1/20...  Training Step: 100...  Training loss: 2.6703...  0.0563 sec/batch
Epoch: 1/20...  Training Step: 150...  Training loss: 2.3466...  0.0591 sec/batch
Epoch: 1/20...  Training Step: 200...  Training loss: 2.2979...  0.0558 sec/batch
Epoch: 1/20...  Training Step: 250...  Training loss: 2.1742...  0.0553 sec/batch
Epoch: 1/20...  Training Step: 300...  Training loss: 2.1130...  0.0557 sec/batch
Epoch: 1/20...  Training Step: 350...  Training loss: 2.0644...  0.0576 sec/batch
Epoch: 1/20...  Training Step: 400...  Training loss: 2.0259...  0.0553 sec/batch
Epoch: 1/20...  Training Step: 450...  Training loss: 2.0517...  0.0558 sec/batch
Epoch: 1/20...  Training Step: 500...  Training loss: 2.2231...  0.0557 sec/batch
Epoch: 1/20...  Training Step: 550...  Training loss: 1.9954...  0.0548 sec/batch
Epoch: 1/20...  Training Step: 600...  Training loss: 2.1538...  0.0576 sec/batch
Epoch: 1/20...  T

Epoch: 2/20...  Training Step: 5000...  Training loss: 1.5436...  0.0585 sec/batch
Epoch: 2/20...  Training Step: 5050...  Training loss: 1.5837...  0.0548 sec/batch
Epoch: 2/20...  Training Step: 5100...  Training loss: 1.7119...  0.0545 sec/batch
Epoch: 2/20...  Training Step: 5150...  Training loss: 1.6981...  0.0552 sec/batch
Epoch: 2/20...  Training Step: 5200...  Training loss: 1.6365...  0.0584 sec/batch
Epoch: 2/20...  Training Step: 5250...  Training loss: 1.7105...  0.0549 sec/batch
Epoch: 2/20...  Training Step: 5300...  Training loss: 1.7501...  0.0579 sec/batch
Epoch: 2/20...  Training Step: 5350...  Training loss: 1.7755...  0.0550 sec/batch
Epoch: 2/20...  Training Step: 5400...  Training loss: 1.6822...  0.0554 sec/batch
Epoch: 2/20...  Training Step: 5450...  Training loss: 1.6920...  0.0583 sec/batch
Epoch: 2/20...  Training Step: 5500...  Training loss: 1.6102...  0.0552 sec/batch
Epoch: 2/20...  Training Step: 5550...  Training loss: 1.8790...  0.0562 sec/batch
Epoc

Epoch: 3/20...  Training Step: 9950...  Training loss: 1.6267...  0.0559 sec/batch
Epoch: 3/20...  Training Step: 10000...  Training loss: 1.4471...  0.0565 sec/batch
Epoch: 3/20...  Training Step: 10050...  Training loss: 1.6728...  0.0544 sec/batch
Epoch: 3/20...  Training Step: 10100...  Training loss: 1.5304...  0.0565 sec/batch
Epoch: 3/20...  Training Step: 10150...  Training loss: 1.6472...  0.0576 sec/batch
Epoch: 3/20...  Training Step: 10200...  Training loss: 1.4948...  0.0546 sec/batch
Epoch: 3/20...  Training Step: 10250...  Training loss: 1.5434...  0.0557 sec/batch
Epoch: 3/20...  Training Step: 10300...  Training loss: 1.6984...  0.0560 sec/batch
Epoch: 3/20...  Training Step: 10350...  Training loss: 1.5468...  0.0570 sec/batch
Epoch: 3/20...  Training Step: 10400...  Training loss: 1.6139...  0.0552 sec/batch
Epoch: 3/20...  Training Step: 10450...  Training loss: 1.5830...  0.0552 sec/batch
Epoch: 3/20...  Training Step: 10500...  Training loss: 1.3385...  0.0567 sec

Epoch: 4/20...  Training Step: 14850...  Training loss: 1.6192...  0.0575 sec/batch
Epoch: 4/20...  Training Step: 14900...  Training loss: 1.5989...  0.0561 sec/batch
Epoch: 4/20...  Training Step: 14950...  Training loss: 1.4376...  0.0579 sec/batch
Epoch: 4/20...  Training Step: 15000...  Training loss: 1.4295...  0.0598 sec/batch
Epoch: 4/20...  Training Step: 15050...  Training loss: 1.6020...  0.0552 sec/batch
Epoch: 4/20...  Training Step: 15100...  Training loss: 1.5742...  0.0563 sec/batch
Epoch: 4/20...  Training Step: 15150...  Training loss: 1.7546...  0.0565 sec/batch
Epoch: 4/20...  Training Step: 15200...  Training loss: 1.5145...  0.0560 sec/batch
Epoch: 4/20...  Training Step: 15250...  Training loss: 1.4862...  0.0579 sec/batch
Epoch: 4/20...  Training Step: 15300...  Training loss: 1.6045...  0.0552 sec/batch
Epoch: 4/20...  Training Step: 15350...  Training loss: 1.6473...  0.0597 sec/batch
Epoch: 4/20...  Training Step: 15400...  Training loss: 1.5748...  0.0573 se

Epoch: 6/20...  Training Step: 19750...  Training loss: 1.5343...  0.0588 sec/batch
Epoch: 6/20...  Training Step: 19800...  Training loss: 1.5268...  0.0571 sec/batch
Epoch: 6/20...  Training Step: 19850...  Training loss: 1.6143...  0.0554 sec/batch
Epoch: 6/20...  Training Step: 19900...  Training loss: 1.5606...  0.0554 sec/batch
Epoch: 6/20...  Training Step: 19950...  Training loss: 1.4928...  0.0561 sec/batch
Epoch: 6/20...  Training Step: 20000...  Training loss: 1.5151...  0.0565 sec/batch
Epoch: 6/20...  Training Step: 20050...  Training loss: 1.4127...  0.0585 sec/batch
Epoch: 6/20...  Training Step: 20100...  Training loss: 1.6009...  0.0618 sec/batch
Epoch: 6/20...  Training Step: 20150...  Training loss: 1.6149...  0.0580 sec/batch
Epoch: 6/20...  Training Step: 20200...  Training loss: 1.5549...  0.0556 sec/batch
Epoch: 6/20...  Training Step: 20250...  Training loss: 1.5648...  0.0634 sec/batch
Epoch: 6/20...  Training Step: 20300...  Training loss: 1.4810...  0.0590 se

...

Epoch: 15/20...  Training Step: 58700...  Training loss: 1.5565...  0.0593 sec/batch
Epoch: 15/20...  Training Step: 58750...  Training loss: 1.4726...  0.0545 sec/batch
Epoch: 15/20...  Training Step: 58800...  Training loss: 1.4348...  0.0598 sec/batch
Epoch: 15/20...  Training Step: 58850...  Training loss: 1.5684...  0.0564 sec/batch
Epoch: 15/20...  Training Step: 58900...  Training loss: 1.5185...  0.0547 sec/batch
Epoch: 15/20...  Training Step: 58950...  Training loss: 1.4925...  0.0576 sec/batch
Epoch: 16/20...  Training Step: 59000...  Training loss: 1.4395...  0.0556 sec/batch
Epoch: 16/20...  Training Step: 59050...  Training loss: 1.5038...  0.0585 sec/batch
Epoch: 16/20...  Training Step: 59100...  Training loss: 1.4266...  0.0572 sec/batch
Epoch: 16/20...  Training Step: 59150...  Training loss: 1.6464...  0.0562 sec/batch
Epoch: 16/20...  Training Step: 59200...  Training loss: 1.4901...  0.0587 sec/batch
Epoch: 16/20...  Training Step: 59250...  Training loss: 1.3641..

Epoch: 17/20...  Training Step: 63550...  Training loss: 1.3300...  0.0562 sec/batch
Epoch: 17/20...  Training Step: 63600...  Training loss: 1.4263...  0.0555 sec/batch
Epoch: 17/20...  Training Step: 63650...  Training loss: 1.3875...  0.0616 sec/batch
Epoch: 17/20...  Training Step: 63700...  Training loss: 1.4580...  0.0582 sec/batch
Epoch: 17/20...  Training Step: 63750...  Training loss: 1.3507...  0.0582 sec/batch
Epoch: 17/20...  Training Step: 63800...  Training loss: 1.4733...  0.0564 sec/batch
Epoch: 17/20...  Training Step: 63850...  Training loss: 1.5401...  0.0561 sec/batch
Epoch: 17/20...  Training Step: 63900...  Training loss: 1.3479...  0.0559 sec/batch
Epoch: 17/20...  Training Step: 63950...  Training loss: 1.3774...  0.0589 sec/batch
Epoch: 17/20...  Training Step: 64000...  Training loss: 1.5295...  0.0560 sec/batch
Epoch: 17/20...  Training Step: 64050...  Training loss: 1.4450...  0.0571 sec/batch
Epoch: 17/20...  Training Step: 64100...  Training loss: 1.5648..

Epoch: 18/20...  Training Step: 68400...  Training loss: 1.5332...  0.0560 sec/batch
Epoch: 18/20...  Training Step: 68450...  Training loss: 1.4396...  0.0553 sec/batch
Epoch: 18/20...  Training Step: 68500...  Training loss: 1.3572...  0.0558 sec/batch
Epoch: 18/20...  Training Step: 68550...  Training loss: 1.6894...  0.0563 sec/batch
Epoch: 18/20...  Training Step: 68600...  Training loss: 1.5294...  0.0566 sec/batch
Epoch: 18/20...  Training Step: 68650...  Training loss: 1.5017...  0.0588 sec/batch
Epoch: 18/20...  Training Step: 68700...  Training loss: 1.4366...  0.0558 sec/batch
Epoch: 18/20...  Training Step: 68750...  Training loss: 1.7005...  0.0571 sec/batch
Epoch: 18/20...  Training Step: 68800...  Training loss: 1.4160...  0.0547 sec/batch
Epoch: 18/20...  Training Step: 68850...  Training loss: 1.5413...  0.0635 sec/batch
Epoch: 18/20...  Training Step: 68900...  Training loss: 1.4444...  0.0656 sec/batch
Epoch: 18/20...  Training Step: 68950...  Training loss: 1.3139..

Epoch: 19/20...  Training Step: 73250...  Training loss: 1.4571...  0.0569 sec/batch
Epoch: 19/20...  Training Step: 73300...  Training loss: 1.5485...  0.0606 sec/batch
Epoch: 19/20...  Training Step: 73350...  Training loss: 1.3818...  0.0558 sec/batch
Epoch: 19/20...  Training Step: 73400...  Training loss: 1.3967...  0.0557 sec/batch
Epoch: 19/20...  Training Step: 73450...  Training loss: 1.4523...  0.0557 sec/batch
Epoch: 19/20...  Training Step: 73500...  Training loss: 1.6291...  0.0620 sec/batch
Epoch: 19/20...  Training Step: 73550...  Training loss: 1.6411...  0.0553 sec/batch
Epoch: 19/20...  Training Step: 73600...  Training loss: 1.4804...  0.0570 sec/batch
Epoch: 19/20...  Training Step: 73650...  Training loss: 1.5207...  0.0569 sec/batch
Epoch: 19/20...  Training Step: 73700...  Training loss: 1.4863...  0.0579 sec/batch
Epoch: 19/20...  Training Step: 73750...  Training loss: 1.4568...  0.0569 sec/batch
Epoch: 19/20...  Training Step: 73800...  Training loss: 1.3614..

Epoch: 20/20...  Training Step: 78100...  Training loss: 1.3930...  0.0576 sec/batch
Epoch: 20/20...  Training Step: 78150...  Training loss: 1.5480...  0.0561 sec/batch
Epoch: 20/20...  Training Step: 78200...  Training loss: 1.4588...  0.0546 sec/batch
Epoch: 20/20...  Training Step: 78250...  Training loss: 1.4167...  0.0587 sec/batch
Epoch: 20/20...  Training Step: 78300...  Training loss: 1.3109...  0.0560 sec/batch
Epoch: 20/20...  Training Step: 78350...  Training loss: 1.5060...  0.0575 sec/batch
Epoch: 20/20...  Training Step: 78400...  Training loss: 1.4339...  0.0578 sec/batch
Epoch: 20/20...  Training Step: 78450...  Training loss: 1.4630...  0.0545 sec/batch
Epoch: 20/20...  Training Step: 78500...  Training loss: 1.6100...  0.0582 sec/batch
Epoch: 20/20...  Training Step: 78550...  Training loss: 1.4924...  0.0583 sec/batch
Epoch: 20/20...  Training Step: 78600...  Training loss: 1.2984...  0.0554 sec/batch


#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [98]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i78640_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i59000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i59200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i59400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i59600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i59800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i62000_l128.ckpt"
al

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [99]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [100]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [101]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i78640_l128.ckpt'

In [102]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

Far, works, said of him, with the more wearing of this all.  "What had thinked it with the same and thought."  "Why do it's never convinted that he had to the present, and I dance too men atting that I was a come, a stop the crossing to see, through as and trious all she set all a still saying at all time to tell that through that's the sappiciator. I did the wife of hands of set out to the soficiar on the carming to make the thing in a meaning. I should be alone to the sale in the thing to suddenly a colleth, into an eager ashack of them with is is." Levin who were all any feet in the particularly.  "It's something that in myself. Who has nothing, I asked as to see there out to be set that the priest to think, and all he can are there was all the sense, but and trying to shall not be both of the sense, and the children, and we'll see to see," said Stepan Arkadin all the choice with a more with his wife strace a take of the course.  "I should be contented it, but into it, a so stairs w

In [104]:
checkpoint = 'checkpoints/i65000_l128.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Far of the chest or that the sound over her hands asked how the side that hurry, where her said and seemed that a thought and when he was any side of a listless serted of the tone of her, and he held the same official prayer he had not can to stroigh its some simple. Have sat down at those seeing it to the servant.  "If yes."  "You don't know his sour wishes as there's some men at marriages what I'm not seemed it!" said Vronsky; and the self-both at time. He could not have a porter and at his shoulders, show that's anything, and they was nice on serious shating with a loving all that he set out of in all.  The same that he was so see than so men, that the self and something of the meadow which already his from a same something he had seemed the senters.  "I want over a carriage and have to go the minute, but I did not late her in the mire, but though she was to came it. It was to can most almust and that it was the more of a son that you wish," said Alexey Alexandrovitch, gazed to and 

In [105]:
checkpoint = 'checkpoints/i72200_l128.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Are")
print(samp)

Arestits she went out of his heart, were the sense.  "I want an everyording than thirks? I don't can all her to her, between at these old moment wouldn't see my simpl oully back of it, what despeases to be so at him to the much would be a streel as that in his to me, and I do they have no decides they?" she went outs of the person at the seas of the sense. He wing to herself of his wife he was never come of a little talk with the three steps of the path as their husband all and a merrer, and he changed the string or the summer. "What's same to me to meet him."  And with a complecter, was so to the pirtoring of him why he would her, he should tortied, and the servate, standing at the men his walk of his brother of the cordy of the command.  "What do I could be something of her,aness in the their sort to him in things."  "I don't bely the sense, and there are all the thousand of them on a much on the child, and an is all that a so and his that seeing."  Levin took him to her a shes, to h

In [109]:
checkpoint = 'checkpoints/i60000_l128.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Life")
print(samp)

Lifer, and to coming on the same time. "Well, who, you? If I say," she asked to them, said his head that he closed a cried of hat to all that he was him to high that it were temp the can hand, and high to the temper out to the servantion. And talking to him to them of his sones at hiss. "Ah, what do! A commension. It time of his sares through, but you were beances we can back her in the crast and her, that is they has all off, all all, which that?" he said, and sent and huddering he say, a simplic wonder this he were a told by her sought of stepper of a stopper. "We live him, and, I were so still the matter to the conversation, when there's nothing?" he asked her fruening and cares women her face, and had.  "It went." He saw her hind with her for this, and she had say, that he was telling to the carriage were at sorry of the same to him and, the part and say, and when his family, but all three considerarys to all with his sallary had his friend that he with the position of her band wer